### 用 ChatGPT API 打造歌詞生成器，並且同時利用 Dall-E 3 生成專輯圖片


In [1]:
!pip install openai
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 7.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 2.8 MB/s eta 0:00

In [2]:
from openai import OpenAI
import gradio as gr

/Users/tvbs/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/tvbs/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_key = "./python-work/api_key.txt"
model = "gpt-3.5-turbo"

In [4]:
client = OpenAI(
    api_key=api_key,
)

In [5]:
character = '''你是一個歌詞生成機器人，依據使用者給的內容，生成出歌詞，生成的歌詞可以頭尾呼應，如果可以押韻的話更好。

請以歌詞生成機器人與使用者開始對談

'''
description = "嗨！我是你的歌詞生成機器人！很高興能和你一起創作歌詞，你想要我們創作什麼樣的歌詞呢？除了創作歌詞外，我還能幫你一起繪出適合這首歌的專輯圖片！"

In [6]:
messages = [{"role":"system",
             "content":character},
            {"role":"assistant",
            'content':description}]

In [7]:
def pipi(prompt, history):
    history = history or []
    global messages

    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content

    # 使用 Dall-E 3 將回應生成圖片
    res = client.images.generate(
        model='dall-e-3',
        prompt= f"依據{reply}生成一張適合的專輯封面", # 依據問題回覆來生成圖片
        n=1, # 生成圖片張數
        quality='hd', # 影像畫質
        size='1024x1024', # 影像大小
        style='natural', # 風格
    )
    image = res.data[0].url
    messages.append({"role": "assistant", "content": reply})
    history = history + [[prompt, reply]]

    return history, history, image

In [8]:
chatbot = gr.Chatbot()

In [9]:
iface = gr.Interface(pipi,
                     inputs=["text", "state"],
                     outputs=[chatbot, "state",gr.Image()],
                     title="歌詞生成機器人",
                     description=description)

In [10]:
iface.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://775e053674c53855cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://775e053674c53855cb.gradio.live
